<a href="https://colab.research.google.com/github/joaochenriques/IST---Marine-Currents-and-Tidal-Energy/blob/main/Tidal_Stream/DiskActuator/ChannelFlows_PowerConversionLimits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.optimize import root_scalar
import subprocess
import pathlib

def cmdcall( cmd, verbose = False ):
    output = subprocess.getoutput( cmd )
    if verbose: print(output)

if not pathlib.Path("mpl_utils.py").exists():
  cmdcall( "curl -O https://raw.githubusercontent.com/joaochenriques/ipynb_libs/main/mpl_utils.py" )

import matplotlib.pyplot as mpl
import mpl_utils as mut
mut.config_plots()

from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')

mpl.rcParams["figure.figsize"] = (12.5,3)

<img src="https://raw.githubusercontent.com/joaochenriques/IST_MCTE/main/ChannelFlows/DiskActuator/Figures/sub_domains.svg" width="500px" style="display:inline">

In [ ]:
fig, (ax1) = mpl.subplots(1,1 )
fig.subplots_adjust( wspace = .22 )

C_T_lst = []
C_P_lst = []

for alpha in np.linspace( 1.001, 3, 100 ):

    u4b = alpha * u1
    h4 = E1 - u4b**2 / ( 2 * gr )

    kc = u1**2 * h1 + 0.5 * gr * ( h1**2 - h4**2 )
    kb = 0.5 * B * h1

    k1 = q - h4 * u4b

    u4t = ( k1 + np.sqrt( k1**2 + 4*kb*( u4b*( q + kb*u4b) - kc ) ) ) / ( 2 * kb )
    h4b = ( q - h4 * u4t ) / ( u4b - u4t )
    h4t = h4 - h4b

    u2t = u4t * h4t / ( h1 * B )

    C_T = ( u4b**2 - u4t**2 ) / u1**2
    C_P = C_T * u2t / u1

    # print( f'h1 = {h1:.5f}, h4 = {h4:.5f}, h4b = {h4b:.5f}, h4t = {h4t:.5f}, '\
    #       f'u1 = {u1:.5f}, u4t = {u4t:.5f}, u4b = {u4b:.5f}, ' \
    #       f'CT = {C_T:.5f}, CP = {C_P:.5f}' )

    if C_P <= 0.0: break

    C_P_lst.append( C_P )
    C_T_lst.append( C_T )

ax1.set_title( "B=%.2f" % B )
ax1.plot( C_T_lst, C_P_lst, label="$\mathrm{Fr}_1=%.2f$" % Fr1 )
ax1.set_ylabel( "$C_P$" )
ax1.grid()
ax1.legend(loc="lower center");
#ax1.set_xticklabels( [] )
ax1.set_xlabel( "$C_T$" )

mpl.savefig('ChannelFlowLimits_Ex.pdf', bbox_inches='tight', pad_inches=0.02);

In [ ]:
def CardanoRoots( aa, bb ):
  # Cardano algorithm to solve our polynomial, see:
  # https://www.shsu.edu/kws006/professional/Concepts_files/SolvingCubics.pdf
  P = -2.0*aa
  Q = -2.0*bb
  Δ = (P/3.0)**3 + (Q/2)**2
  if Δ < 0.0: Δ = Δ + 0J
  β = ( -Q/2.0 - np.sqrt(Δ) )**(1.0/3.0)
  α = P/(3.0*β)
  ω = ( -1.0 + np.sqrt(3.0)*1J) / 2.0

  x1 = α - β
  x2 = (α*ω - β)*ω
  x3 = (α - β*ω)*ω

  if np.imag(x1) < 1E-15: x1 = np.real( x1 )
  if np.imag(x2) < 1E-15: x2 = np.real( x2 )
  if np.imag(x3) < 1E-15: x3 = np.real( x3 )

  # applies only for this solution
  assert( np.imag( x1 ) == 0 )
  assert( np.imag( x2 ) == 0 )
  assert( np.imag( x3 ) == 0 )
  assert( x1 <= 0.0 )
  assert( x2 <= x3 )

  return (x2, x3)